In [51]:
import pandas as pd
import numpy as np
from pulp import *

In [52]:
facility=["R","H","M","W"]
facility_numbers=[173,638,169,202]
Demand=[150,188,547,159]
Demand_locations=['Wurzburg','Duren','Remscheid','Nurnberg']

In [53]:
values=np.array([[76,63,77,64],
                [96,86,90,78],
                [80,63,70,57],
                [66,88,70,98]])

In [54]:
df=pd.DataFrame(index=facility,data=values,columns=[i for i in Demand_locations])
print df

   Wurzburg  Duren  Remscheid  Nurnberg
R        76     63         77        64
H        96     86         90        78
M        80     63         70        57
W        66     88         70        98


In [55]:
locations_demand=dict(zip(Demand_locations,Demand))
supply_values=dict(zip(facility,facility_numbers))
print locations_demand['Wurzburg']
fare_in_order=[df.ix['R'],df.ix['H'],df.ix['M'],df.ix['W']]
fare_combos=makeDict([facility,Demand_locations],fare_in_order)
print fare_combos

150
{'H': {'Duren': 86, 'Wurzburg': 96, 'Remscheid': 90, 'Nurnberg': 78}, 'R': {'Duren': 63, 'Wurzburg': 76, 'Remscheid': 77, 'Nurnberg': 64}, 'M': {'Duren': 63, 'Wurzburg': 80, 'Remscheid': 70, 'Nurnberg': 57}, 'W': {'Duren': 88, 'Wurzburg': 66, 'Remscheid': 70, 'Nurnberg': 98}}


In [49]:
#formulate the problem
prob =LpProblem("Cake",LpMinimize)

#define the decision variables
x=LpVariable.dicts("dec_vars",(facility,Demand_locations),lowBound=0,upBound=None)
#this creates decisionvariables in the form 
#x['H']['Wurzburg'] so using indexes all dec variables can be accessed

#define the objective function
prob +=lpSum(x[i][j]*fare_combos[i][j] for i in facility for j in Demand_locations)

#constraints
#demand constraint
for i in Demand_locations:
    prob+=lpSum(x[j][i] for j in facility)>=locations_demand[i]
    
#supply constraint
for i in facility:
    prob+=lpSum(x[i])<=supply_values[i]

    
# solve the problem
status = prob.solve()
LpStatus[status]

'Optimal'

In [50]:
# print the results
print (value(prob.objective))
print ('\n')

print(prob)
for v in prob.variables():
    print(v.name, "=", v.varValue)

79102.0


Cake:
MINIMIZE
86*dec_vars_H_Duren + 78*dec_vars_H_Nurnberg + 90*dec_vars_H_Remscheid + 96*dec_vars_H_Wurzburg + 63*dec_vars_M_Duren + 57*dec_vars_M_Nurnberg + 70*dec_vars_M_Remscheid + 80*dec_vars_M_Wurzburg + 63*dec_vars_R_Duren + 64*dec_vars_R_Nurnberg + 77*dec_vars_R_Remscheid + 76*dec_vars_R_Wurzburg + 88*dec_vars_W_Duren + 98*dec_vars_W_Nurnberg + 70*dec_vars_W_Remscheid + 66*dec_vars_W_Wurzburg + 0
SUBJECT TO
_C1: dec_vars_H_Wurzburg + dec_vars_M_Wurzburg + dec_vars_R_Wurzburg
 + dec_vars_W_Wurzburg >= 150

_C2: dec_vars_H_Duren + dec_vars_M_Duren + dec_vars_R_Duren + dec_vars_W_Duren
 >= 188

_C3: dec_vars_H_Remscheid + dec_vars_M_Remscheid + dec_vars_R_Remscheid
 + dec_vars_W_Remscheid >= 547

_C4: dec_vars_H_Nurnberg + dec_vars_M_Nurnberg + dec_vars_R_Nurnberg
 + dec_vars_W_Nurnberg >= 159

_C5: dec_vars_R_Duren + dec_vars_R_Nurnberg + dec_vars_R_Remscheid
 + dec_vars_R_Wurzburg <= 173

_C6: dec_vars_H_Duren + dec_vars_H_Nurnberg + dec_vars_H_Remscheid
 + dec_vars_H